
# Building a Simple AI Agent (Parts 1–3)

This notebook combines **Part 1**, **Part 2**, and **Part 3** of the tutorial into a single, executable document.  
You'll learn how to:
- Structure an **agent loop**
- **Parse** model responses
- **Execute** tool actions
- **Update memory**
- Decide when to **terminate**

> The examples below include a minimal, runnable mock Agent that uses simple tools (`list_files`, `read_file`) and a mock `generate_response` function to simulate an LLM.



## Part 1 — The Agent Loop in Python

The agent loop is the backbone of our AI agent, enabling it to perform tasks by combining response generation, action execution, and memory updates in an iterative process.

### Steps in the Agent Loop

1. **Construct Prompt**: Combine the agent’s memory, user input, and system rules into a single prompt.  
2. **Generate Response**: Send the constructed prompt to the LLM and retrieve a response.  
3. **Parse Response**: Extract the intended action and its parameters from the LLM’s output.  
4. **Execute Action**: Perform the requested task with the appropriate tool.  
5. **Convert/Store Result**: Save the result and feedback to memory.  
6. **Continue Loop?**: Terminate if instructed or if a stopping condition is reached.



### Agent Loop (Conceptual Example)
```python
# The Agent Loop
while iterations < max_iterations:

    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)

    result = "Action executed"

    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break

    iterations += 1
```



### Constructing the Agent Prompt

The prompt is created by appending the agent’s rules (system message) to the current memory of interactions:

```python
prompt = agent_rules + memory
```



### Agent Rules: Defining the Agent’s Behavior

Below is a sample `agent_rules` structure included at the start of every iteration to guide the agent.


In [1]:

agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:
- list_files() -> List[str]: List all files in the current directory.
- read_file(file_name: str) -> str: Read the content of a file.
- terminate(message: str): End the agent loop and print a summary to the user.

If a user asks about files, list them before reading.

Every response MUST have an action.
Respond in this format:

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```
"""
}]
print("Agent rules initialized.")

Agent rules initialized.



## Part 2 — Interfacing with the Environment

Once the Agent has generated a response, we interpret it as an **action** to execute in the environment.



### Step 3: Parse the Response

The model is expected to respond with a JSON action inside a markdown code block labeled `action`.  
We extract that JSON and parse it into a Python dictionary.


In [2]:

import json
import re
from typing import Dict, Any, List

def extract_markdown_block(text: str, fence: str) -> str:
    """Extract the content of a fenced code block like ```action ... ```"""
    # Regex to capture content inside ```action ... ``` fences
    pattern = re.compile(rf"""```{re.escape(fence)}\s*(.*?)\s*```""", re.DOTALL)
    m = pattern.search(text)
    if not m:
        raise ValueError(f"No fenced block labeled '{fence}' found.")
    return m.group(1)

def parse_action(response: str) -> Dict[str, Any]:
    """Parse the LLM response into a structured action dictionary."""
    try:
        payload = extract_markdown_block(response, "action")
        response_json = json.loads(payload)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except ValueError:
        return {"tool_name": "error", "args": {"message": "No action block found. Respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

print("Parsing utilities ready.")

Parsing utilities ready.



### Step 4: Execute the Action

Each `tool_name` maps to a Python function. Below we define a tiny toolset.


In [3]:

import os
from pathlib import Path

# --- Demo setup: create a few small files ---
Path("demo_files").mkdir(exist_ok=True)
with open("demo_files/file1.txt", "w", encoding="utf-8") as f:
    f.write("Hello from file1. This is a demo file.")

with open("demo_files/file2.txt", "w", encoding="utf-8") as f:
    f.write("Greetings from file2. Another demo content.")

def list_files() -> List[str]:
    return sorted([p.name for p in Path("demo_files").glob("*") if p.is_file()])

def read_file(file_name: str) -> str:
    p = Path("demo_files") / file_name
    if not p.exists() or not p.is_file():
        return f"File '{file_name}' not found."
    return p.read_text(encoding="utf-8")

print("Tools ready. Demo files created in ./demo_files")

Tools ready. Demo files created in ./demo_files



## Part 3 — Memory & Termination



### Step 5: Update the Agent’s Memory

After executing an action, we append both the agent’s intent (the LLM response) and the result of the action to memory.



### Step 6: Decide Whether to Continue

The loop terminates if it receives a `terminate` action or a stopping condition (like `max_iterations`) is met.



## End-to-End Demo

Below is a **fully runnable** mini-agent demonstrating the entire loop with a **mock** `generate_response` function that simulates an LLM.  
The mock policy is simple:
- If we haven't listed files yet, respond with `list_files`.
- If we have listed files but haven't read one, respond with `read_file` for the first file.
- After reading, respond with `terminate` and a short message.


In [ ]:

def generate_response(prompt_messages: List[Dict[str, Any]]) -> str:
    """A tiny mock LLM that decides next action from memory in prompt_messages."""
    # Find last user feedback (which usually contains results) and last assistant action
    last_user = None
    last_assistant_action = None
    for msg in reversed(prompt_messages):
        if msg.get("role") == "user" and last_user is None:
            last_user = msg.get("content")
        if msg.get("role") == "assistant" and last_assistant_action is None:
            last_assistant_action = msg.get("content")
        if last_user and last_assistant_action:
            break

    # Heuristics
    try:
        # If we already got a list of files as user feedback, try to read the first file
        if last_user and last_user.strip().startswith("[") and "file" in last_user:
            files = json.loads(last_user)
            if isinstance(files, list) and files:
                action = {
                    "tool_name": "read_file",
                    "args": {"file_name": files[0]}
                }
                return f"""```action
{json.dumps(action)}
```"""
    except Exception:
        pass

    # If we already read a file, terminate
    if last_assistant_action and "read_file" in last_assistant_action:
        action = {"tool_name": "terminate", "args": {"message": "Finished reading a file. Goodbye!"}}
        return f"""```action
{json.dumps(action)}
```"""

    # Default: list files
    action = {"tool_name": "list_files", "args": {}}
    return f"""```action
{json.dumps(action)}
```"""

# --- Full loop ---
memory: List[Dict[str, Any]] = [
    {"role": "user", "content": "What files are in this directory?"}
]
max_iterations = 5
iterations = 0

while iterations < max_iterations:
    # 1) Construct prompt (rules + memory)
    prompt = agent_rules + memory

    # 2) Generate response (mock LLM)
    print("Agent thinking...")
    response = generate_response(prompt)
    print("Agent response:", response.strip())

    # 3) Parse
    action = parse_action(response)

    # 4) Execute
    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"].get("file_name", ""))}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        # Update memory before breaking, so the transcript is complete
        memory.extend([
            {"role": "assistant", "content": response},
            {"role": "user", "content": json.dumps({"message": action["args"]["message"]})}
        ])
        break
    else:
        result = {"error": "Unknown action: " + str(action.get("tool_name"))}

    print("Action result:", result)

    # 5) Update memory
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6) Check termination (also handled above)
    if action["tool_name"] == "terminate":
        break

    iterations += 1

print("\nFinal memory transcript:")
for m in memory:
    print(m)

Agent thinking...
Agent response: ```action
{"tool_name": "list_files", "args": {}}
```
Action result: {'result': ['file1.txt', 'file2.txt']}
Agent thinking...
Agent response: ```action
{"tool_name": "list_files", "args": {}}
```
Action result: {'result': ['file1.txt', 'file2.txt']}
Agent thinking...
Agent response: ```action
{"tool_name": "list_files", "args": {}}
```
Action result: {'result': ['file1.txt', 'file2.txt']}
Agent thinking...
Agent response: ```action
{"tool_name": "list_files", "args": {}}
```
Action result: {'result': ['file1.txt', 'file2.txt']}
Agent thinking...
Agent response: ```action
{"tool_name": "list_files", "args": {}}
```
Action result: {'result': ['file1.txt', 'file2.txt']}

Final memory transcript:
{'role': 'user', 'content': 'What files are in this directory?'}
{'role': 'assistant', 'content': '```action\n{"tool_name": "list_files", "args": {}}\n```'}
{'role': 'user', 'content': '{"result": ["file1.txt", "file2.txt"]}'}
{'role': 'assistant', 'content': '```


---

_Exported on 2025-09-11 04:36:28 UTC_
